In [1]:
import pandas as pd
import geopandas as gpd

## Filter with the Manhattan boundary

Please do not start this part if you have the `building_manhattan.geojson` file

In [2]:
# import dataset (from https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh)
gdf_building = gpd.read_file('../../data/raw/building/Building Footprints.geojson')

In [ ]:
# filtering buildings with the Manhattan boundary
gdf_manhattan = gpd.read_file('../../data/raw/boundary/Borough Boundaries.geojson')
gdf_manhattan = gdf_manhattan.loc[gdf_manhattan.loc[:,'boro_name'] == 'Manhattan']

gdf_building_manhattan = gpd.sjoin(gdf_building, gdf_manhattan, how='inner', op='intersects')

In [ ]:
gdf_building_manhattan.to_file('../../data/raw/building/building_manhattan.geojson', driver='GeoJSON')

## Calculate the nearest street

In [25]:
gdf_building_manhattan = gpd.read_file('../../data/raw/building/building_manhattan.geojson')
gdf_streets = gpd.read_file('../../data/raw/lion/edges_manhattan.geojson')

gdf_building_manhattan = gdf_building_manhattan.drop(['shape_len','index_right', 'boro_code', 'boro_name', 'shape_area_right','shape_leng'], axis=1)

In [26]:
# unify the coordinates systems
gdf_building_manhattan = gdf_building_manhattan.to_crs(2263)
gdf_streets.crs == gdf_building_manhattan.crs

True

In [27]:
gdf_building_manhattan = gdf_building_manhattan.sjoin_nearest(gdf_streets.loc[:,['OBJECTID','StreetWidth_Min','StreetWidth_Max','POSTED_SPEED','geometry']])

# because duplicated rows created during the spatial join process, I dropped
gdf_building_manhattan = gdf_building_manhattan.drop_duplicates(subset=['base_bbl','mpluto_bbl','globalid']).drop('index_right', axis=1)

In [32]:
gdf_building_manhattan.head()

,name,base_bbl,shape_area_left,heightroof,mpluto_bbl,cnstrct_yr,globalid,lststatype,feat_code,groundelev,geomsource,bin,lstmoddate,doitt_id,geometry,OBJECTID,StreetWidth_Min,StreetWidth_Max,POSTED_SPEED
0,None,1021210037,0.0,59.722628,1021210037,1910,{A0E56BCC-A86B-4CEF-9A42-9B4ECD61743F},Constructed,2100,154,Photogramm,1062896,2017-08-22,708881,"MULTIPOLYGON (((1001211.761 244524.798, 100123...",100007,60.0,60.0,25
1,None,1008870001,0.0,155.49,1008870001,1935,{0B06F7A8-1F83-44F3-97DD-AC524A6374CA},Constructed,2100,37,Photogramm,1018457,2017-08-22,584238,"MULTIPOLYGON (((989135.970 210787.603, 989181....",101046,32.0,32.0,25
2,None,1013490035,0.0,167.21446831,1013490035,1931,{A091951D-C73E-4B77-9A61-716C666C5446},Constructed,2100,52,Photogramm,1039988,2017-08-22,66240,"MULTIPOLYGON (((994153.480 215659.003, 994165....",99804,60.0,60.0,25
3,None,1010610006,0.0,28.57,1010610006,1910,{2BE9F674-67E7-4A49-8535-F636A16D8DA5},Constructed,2100,40,Photogramm,1026714,2017-08-22,23730,"MULTIPOLYGON (((986858.328 218070.430, 986839....",103068,32.0,32.0,25
4,None,1003060013,0.0,73.13,1003060013,1900,{2403088E-77A6-4071-8B5E-D54AFBC720D9},Constructed,2100,39,Photogramm,1003956,2017-08-22,512541,"MULTIPOLYGON (((986180.396 200723.234, 986187....",92425,34.0,45.0,25
